In [11]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import gensim
from gensim.models import word2vec
from collections import defaultdict
import nltk
import re
from spacy.lang.en import English
import chainer
from chainer import datasets
from chainer import functions as F
from chainer import links as L
from chainer import Variable
from chainer.backends import cuda
from sklearn.model_selection import train_test_split

prog = re.compile('\.[A-Z]')
Head_check = re.compile('^[A-Z][^A-Z]+$')
nlp = English()
pd.set_option('display.max_columns', 50)

In [2]:
train_df = pd.read_csv("../input/train.csv")
# test_df = pd.read_csv("../input/test.csv")

In [3]:
len(train_df) + len(test_df)

1902194

In [4]:
all_document = pd.concat([train_df["comment_text"],test_df["comment_text"]])
all_document.shape

(1902194,)

In [6]:
%%time
t = []
for s in train_df['comment_text']:
    t.append(s.split())

CPU times: user 12.5 s, sys: 1.19 s, total: 13.7 s
Wall time: 13.7 s


In [4]:
del_list = [",", ".", "?", "!", "'"]
for i in range(len(t)):
    for del_ele in del_list:
        if del_ele in t[i]:
            t[i].remove(del_ele)

In [7]:
words = {}
for i in range(len(t)):
    for word in t[i]:
        if word not in words:
            words[word] = len(words)

In [8]:
word_len = len(words)
word_len

1670966

In [7]:
t_vec = []
for i in range(len(t)):
    sentence_ids = []
    for word in t[i]:
        sentence_ids.append(words[word])
    sentence_ids = np.array(sentence_ids, dtype="int32")
    t_vec.append(sentence_ids)


In [36]:
len(t_vec)

1804874

In [39]:
max_len = max(list(map(len, t_vec)))
for i in range(len(t_vec)):
    for j in range((max_len-len(t_vec[i]))):
        t_vec[i] = np.append(t_vec[i], -1)

In [22]:
list(map(len, t_vec))
max(list(map(len, t_vec)))

317

In [15]:
MIN_COUNT = 10

In [10]:
model = word2vec.Word2Vec(sentences, window=5, min_count=MIN_COUNT , iter=5, negative=5)

In [131]:
a = model.wv[sentences[0]]
a.shape

(27, 100)

In [77]:
model.wv.most_similar('suck')

[('screw', 0.7846056818962097),
 ('soak', 0.7200046181678772),
 ('blow', 0.71183180809021),
 ('gobble', 0.6828003525733948),
 ('sucking', 0.6570413708686829),
 ('screwed', 0.6553224921226501),
 ('fucked', 0.6482899188995361),
 ('wake', 0.6468736529350281),
 ('lighten', 0.6461282968521118),
 ('snuff', 0.6439306735992432)]

In [78]:
model.save("jigsaw_word2vec")

/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [81]:
model = word2vec.Word2Vec.load("jigsaw_word2vec")

/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [160]:
model.wv[sentences[0]].shape

(27, 100)

In [19]:
class jigsaw_dataset(chainer.dataset.DatasetMixin):
    def __init__(self, train=True, min_count=10, train_size=0.6):
        train_df = pd.read_csv("../input/train.csv")
        
        t = []
        for s in train_df['comment_text']:
            t.append(s.split())
                
        words = {}
        for j in range(len(t)):
            for word in t[j]:
                if word not in words:
                    words[word] = len(words)
        
        t_vec = []
        for j in range(len(t)):
            t_ids = []
            for word in t[j]:
                t_ids.append(words[word])
            t_ids = np.array(t_ids, dtype=np.int32)
            t_vec.append(t_ids)
        
        max_len = max(list(map(len, t_vec)))
        for i in range(len(t_vec)):
            for j in range((max_len-len(t_vec[i]))):
                t_vec[i] = np.append(t_vec[i], -1)
        
        x = np.array(t_vec, dtype=np.int32)
        
        label= np.where(train_df['target']>0.5, 1, 0)
        label = pd.get_dummies(label) # comment out
        y = label.values
        y = y.astype(np.int32)
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(x, y, train_size=train_size, random_state=1)
        self.n_train = len(self.y_train)
        self.n_test = len(self.y_test)
        self.train = train
        self.train_size=train_size
        
        del t, words, t_vec, t_ids, x, y
    
    def __len__(self):
        if self.train:
            return self.n_train
        else:
            return self.n_test
    
    def get_example(self, i):
        
        train = datasets.tuple_dataset.TupleDataset(self.X_train, self.y_train)
        test = datasets.tuple_dataset.TupleDataset(self.X_test, self.y_test)
        
        if self.train:
            return train[i]
        else:
            return test[i]
        

In [29]:
class RNN(chainer.Chain):
    def __init__(self, n_vocab=1670966, n_units=100):
        super(RNN, self).__init__(
            embed = L.EmbedID(n_vocab, n_units),
            l1 = L.LSTM(None, n_units),
            l2 = L.LSTM(None, n_units),
            l3 = L.Linear(None, 2)
        )
    
    def reset_state(self):
        self.l1.reset_state()
        self.l2.reset_state()

    def forward(self, x):
        h0 = self.embed(x)
        h1 = self.l1(F.dropout(h0))
        h2 = self.l2(F.dropout(h1))
        y = F.softmax(self.l3(h2))
        return y


model = RNN()

# gpu_id = 0
# if gpu_id >= 0:
#     model.to_gpu(gpu_id)
    
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [21]:
BATCH_SIZE = 5
MAX_EPOCH = 5

In [22]:
%%time
train = jigsaw_dataset(train_size=0.1)
test = jigsaw_dataset(train_size=0.1)
train_iter = chainer.iterators.SerialIterator(train, BATCH_SIZE)
test_iter = chainer.iterators.SerialIterator(test, BATCH_SIZE, repeat=False, shuffle=False)

/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


CPU times: user 40min 40s, sys: 7.92 s, total: 40min 48s
Wall time: 40min 40s


In [37]:
while train_iter.epoch < MAX_EPOCH:
    train_batch = train_iter.next()
    sentence_train, target_train = chainer.dataset.concat_examples(train_batch)
#     sentence_train, target_train = chainer.dataset.concat_examples(train_batch, gpu_id)
    
    # target_train = [0, 1, ,1 , 0 , 1, 0,...]
    prediction_train = model(sentence_train)
    # prediction_trian = [(0.5, 0.5), (0.2, 0.8)...]
    loss = F.softmax_cross_entropy(prediction_train, target_train)
    
    model.cleargrads()
    loss.backward()
    
    optimizer.update()
    if train_iter.is_new_epoch:
        print('epoch:{:02d} train_loss:{:.04f}'.format(train_iter.epoch, float(loss.array)), end='')
        
        test_losses = []
        test_accuracies = []
        while True:
            test_batch = test_iter.next()
            sentence_test, target_test = chainer.dataset.concat_examples(test_batch)
#             sentence_test, target_test = chainer.dataset.concat_examples(test_batch, gpu_id)
            
            prediction_test = model(sentence_test)
            loss_test = F.mean_squared_error(prediction_test, target_test)
            test_losses.append(loss_test.array)
#             test_losses.append(cuda.to_cpu(loss_test.array))
            
            accuracy = F.accuracy(prediction_test, target_test)
#             accuracy.cuda.to_cpu()
            test_accuracies.append(accuracy.array)
            
            if test_iter.is_new_epoch:
                test_iter.epoch = 0
                test_iter.current_position = 0
                test_iter.is_new_epoch = False
                test_iter._pushed_position = None
                break;
        
        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(np.mean(test_losses), np.mean(test_accuracies)))

variable([[0.4846383  0.5153617 ]
          [0.33302444 0.6669756 ]
          [0.38394952 0.6160505 ]
          [0.39510062 0.6048994 ]
          [0.49786264 0.50213736]])
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


InvalidType: 
Invalid operation is performed in: SoftmaxCrossEntropy (Forward)

Expect: t.ndim == x.ndim - 1
Actual: 2 != 1